In [13]:
:load KrapshDisplays KrapshDagDisplay

In [14]:
:extension DeriveGeneric
:extension FlexibleContexts
:extension OverloadedStrings
:extension GeneralizedNewtypeDeriving
:extension FlexibleInstances
:extension MultiParamTypeClasses

In [15]:
import Spark.Core.Dataset
import Spark.Core.Context
import Spark.Core.Column
import Spark.Core.ColumnFunctions
import Spark.Core.Functions
import Spark.Core.Row
import Spark.Core.Types
import Spark.Core.Try

import qualified Data.Vector as V
import qualified Data.Text as T
import GHC.Generics

In [16]:
myScaler :: Column ref Double -> Column ref Double
myScaler col =
  let cnt = asDouble (countCol col) @@ "count"
      m = sumCol col / cnt @@ "mean"
      centered = col .- m
      stdDev = sumCol (centered * centered) / cnt @@ "std_dev"
  in centered ./ stdDev


In [17]:
:t myScaler

myScaler :: forall ref. Column ref Double -> Column ref Double

In [18]:
let ds = dataset [-1, 1] :: Dataset Double
let c = myScaler (asCol ds)

In [19]:
let ds2 = pack1 c

In [20]:
showGraph ds2

'>

In [21]:
let res = collect (asCol ds2)

In [22]:
conf = defaultConf {
        confEndPoint = "http://10.0.2.2",
        confRequestedSessionName = "col_ops6" }

createSparkSessionDef conf

[Debug] Creating spark session at url: http://10.0.2.2:8081/sessions/col_ops6 @(<unknown>:<unknown> <unknown>:0:0)

In [24]:
exec1Def res

[Debug] executeCommand1: computing observable collect_4dfb2f@org.spark.Collect![double] @(<unknown>:<unknown> <unknown>:0:0)
[Info] Sending computations at url: http://10.0.2.2:8081/computations/col_ops6/1/create @(<unknown>:<unknown> <unknown>:0:0)
[Debug] executeCommand1: Tracked nodes are [(97a94..,NPath(sum_97a943),double,sum_97a943),(a49c1..,NPath(count_a49c15),int,count_a49c15),(645d6..,NPath(count),double,count),(f7767..,NPath(mean),double,mean),(57104..,NPath(localpack_571046),{_1:double _2:double},localpack_571046),(0c718..,NPath(sum_0c7185),double,sum_0c7185),(b26f7..,NPath(std_dev),double,std_dev),(26a9f..,NPath(localpack_26a9f0),{_1:double _2:double},localpack_26a9f0),(4dfb2..,NPath(collect_4dfb2f),[double],collect_4dfb2f)] @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /sum_97a943 running @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /count_a49c15 running @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /coun

In [27]:
import Spark.Core.StructuresInternal(ComputationID(..))
import IHaskell.Display
stats <- computationStatsDef (ComputationID "1")

[Debug] computationStats: stats for ComputationID {unComputationID = "1"} @(<unknown>:<unknown> <unknown>:0:0)
[Debug] Sending computations stats request at url: http://10.0.2.2:8081/computations_status/col_ops6/1/ @(<unknown>:<unknown> <unknown>:0:0)

In [28]:
let ns = statsToExportNodes stats
let c = T.unpack (tfIFrame ns)
Display [html c]

'>